In [5]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
%matplotlib inline

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-wf_yy2r_ because the default path (/tmp/cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


<div class="alert alert-warning">

# Lesson 4: Linear Systems
    
**You learned about spring systems during lecture. We will now use this knowledge to understand how a Slinky changes its shape due to gravity!**
        
    
![Figure](slinky1.jpg)
    
    
Try taking a slinky and suspend it from one side. What happens to its shape?

Today we will model the deformation (change of shape) of a slinky under the effects of gravitational forces. We will break down the slinky into many small springs, creating a mass-spring system as illustrated below:

<img src="slinky_held2.png" width=350>
    

where in the general case, $f_i$ is the applied force and $u_i$ is the corresponding displacement (movement) at mass $m_i$. For this example, the applied force $f_i$ correspond to the gravitational force.

We want to determine the equilibrium condition of this system, such that the balance of the internal forces (resulting from the spring getting deformed) and external forces (applied forces at the masses) is equal to zero. To accomplish this we will follow Hooke's law, which states the spring force needed to deform a spring by an amount $\Delta L$ is given as $−k \Delta L$, where $k$ is the spring's stiffness. Consider the two springs below:
    
    
<img src="two_springs.png" width=300>


The force that spring $k_i$ exerts on mass $m_i$ is given by $-k_i(u_i - u_{i-1})$. In a similar way, the force that spring $k_{i+1}$ exerts on mass $m_i$ is $k_{i+1}(u_{i+1} - u_{i})$. We can combine these two spring forces and the applied force at $m_i$ to obtain the following equation:
    
<img src="force_balance.png" width=300>


$$f_i - k_i(u_i - u_{i-1}) + k_{i+1}(u_{i+1} - u_{i}) = 0 $$
    
If we apply the equilibrium equation to all $N$ masses of the spring system, we get the following system of equations:

$$
\begin{equation}
f_1 - k_1(u_1 - u_{0}) + k_{2}(u_{2} - u_{1}) = 0 \\
f_2 - k_2(u_2 - u_{1}) + k_{3}(u_{3} - u_{2}) = 0 \\
f_3 - k_3(u_3 - u_{2}) + k_{4}(u_{4} - u_{3}) = 0 \\
f_4 - k_4(u_4 - u_{3}) + k_{5}(u_{5} - u_{4}) = 0 \\
f_5 - k_5(u_5 - u_{4})  = 0 \\
\end{equation}
$$

We can assume that $u_0 = 0$, indicating that one end of the slinky is held in place. The above equations can be re-written in matrix form:

$$
\begin{eqnarray}
\begin{bmatrix}
k_1+k_2 & -k_2 & 0 & 0 & 0 \\
 -k_2 & k_2+k_3 & -k_3 & 0 & 0 \\
 0 & -k_3 & k_3+k_4 & -k_4 & 0 \\
 0 & 0 & -k_4 & k_4 + k_5 & -k_5 \\
 0 & 0 & 0 &  -k_5 & k_5 \\
\end{bmatrix} 
\begin{bmatrix}
u_1\\ u_2 \\ u_3 \\ u_4 \\ u_5
\end{bmatrix} = 
\begin{bmatrix}
 f_1\\ f_2 \\ f_3 \\ f_4 \\ f_5
\end{bmatrix}
\end{eqnarray}
$$

which we can solve for the unknown displacements $u_i$ by solving

$${\bf K}{\bf u} = {\bf f}$$

where we will denote ${\bf K}$ the stiffness matrix, ${\bf u}$ the displacement vector and ${\bf f}$ the force vector.

**We are done! We can now analyze the slinky model.**

</div>

### Problem definition - Input variables:

The overall slinky has mass $M$, length $L_s$ and stiffness $K_s$ (a parameter that depends on the material and the dimensions of the slinky). We will use the parameters below for the metal slinky:

In [6]:
Ls = 10   # (cm)       Length of the slinky
Ks = 700  # (N/cm)     Stiffness
M  = 170  # (grams)    Total mass

Our model assumes that the mass $M$ is split into a series of $(N+1)$ smaller equal masses $m = M/(N+1)$ which are connected by $N$ massless ideal springs. Hence, the slinky  is divided into smaller but stiffer springs, with $l = L_s/N$ and $k = K_s\cdot N$ (in this example, all springs have the same stiffness, since the slinky is made of one material).

In other words, we will **discretize** our model into $N$-many springs. We will start by using $N = 5$. You will later use a finer discretization by increasing the value of $N$.

In [7]:
# Number of springs
N = 5

In [8]:
m = M/(N+1) # each individual mass
k = Ks*N    # each individiual spring stiffness
l = Ls/N    # each individual spring length
m

28.333333333333332

For the spring system above, all the springs will have the same spring stiffness $k$, since we are assuming the entire slinky is made of the same material. But in general, we could combine two slinkies of different materials. For that reason, we will define the array `k_array` which contains each individual spring stiffness:

$$ \textrm{k_array} = [k_1, k_2, k_3, ... , k_N ] $$

In [11]:
k_array = k*np.ones(N)
print(k_array)

[3500. 3500. 3500. 3500. 3500.]


### Obtaining the stiffness matrix

We would like to create the stiffness matrix in a programmatic way. That way, we can later add many more springs, or use different material without having to change our code. We will take advantage of the pattern of this matrix. Here is the stiffness matrix for the 5 spring system:

$$
\begin{eqnarray}
{\bf K} = \begin{bmatrix}
k_1+k_2 & -k_2 & 0 & 0 & 0 \\
 -k_2 & k_2+k_3 & -k_3 & 0 & 0 \\
 0 & -k_3 & k_3+k_4 & -k_4 & 0 \\
 0 & 0 & -k_4 & k_4 + k_5 & -k_5 \\
 0 & 0 & 0 &  -k_5 & k_5 \\
\end{bmatrix}
\end{eqnarray}
$$

**Can you see a pattern?**

Note that each spring contributes to the stiffness matrix with the following sub-matrix:

$$
\begin{eqnarray}
\begin{bmatrix}
k_i & -k_i  \\
 -k_i & k_i\\
\end{bmatrix}
\end{eqnarray}
$$
with the exception of $k_1$.

Once you can recognize the pattern, you will need to:

1) add this sub-matrix in the correct location of the bigger stiffness matrix ${\bf K}$

2) adjust the first line of the stiffness matrix ${\bf K}$, that follows a different pattern due to the boundary conditions

To help you with 1), you could use the following method to insert the sub-matrix `Asub` in the matrix `A` given the position of the entry `Asub[0,0]`. This is not a requirement, there are other ways to accomplish the same thing :-)

In [10]:
A = np.zeros((5,5))
Asub = np.ones((2,2))

# Use Python slicing
# Insert by overriding a sub-matrix
A[0:2,1:3] = Asub
print(A)
# Insert by adding a sub-matrix
A[1:3, 0:2] += Asub
print(A)

[[0. 1. 1. 0. 0.]
 [0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 1. 1. 0. 0.]
 [1. 2. 1. 0. 0.]
 [1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


NameError: name 'i' is not defined

**Check your answers!**  
#### Create the stiffness matrix

Define the function `get_stiffness` that takes the array `k_array` with each spring stiffness, and returns the stiffness matrix ${\bf K}$.  You can refer back to the definition of the stiffness matrix above. Remember that you have to exclude the first $k_1$ in your loop.

In [20]:
#grade (enter your code in this cell - DO NOT DELETE THIS LINE) 
def get_stiffness(k_array):
    # k_array: 1d numpy array with all the stiffness spring values
    # number of springs
    N = len(k_array)
    # create and return an N x N stiffness matrix
    K = np.zeros((N,N))
    for i in range (N-1):
        #A = np.zeros((5,5)) "K"
        Asub = np.array([[k_array[i], -k_array[i]],[-k_array[i], k_array[i]]])

        # Use Python slicing
        # Insert by overriding a sub-matrix

        # Insert by adding a sub-matrix
        K[i:i+2, i:i+2] += Asub
    
    K[0][0] += k_array[0]
    
    return K

Now you can get the stiffness matrix `K` given `k_array`:

In [21]:
K = get_stiffness(k_array)
print(K)

[[ 7000. -3500.     0.     0.     0.]
 [-3500.  7000. -3500.     0.     0.]
 [    0. -3500.  7000. -3500.     0.]
 [    0.     0. -3500.  7000. -3500.]
 [    0.     0.     0. -3500.  3500.]]


For the slinky model, the applied forces correspond to the gravitational forces, as such we evaluate:

$$f_i = m_i g$$

**Check your answers!**  

Recall that in this model, we assume that the mass of the slinky $M$ is distributed equally in smaller equal masses $m_i = m$. You need to create the right-hand side, the vector ${\bf f}$ where each entry is given as $f_i = m_i g$. Define the function `get_force` that takes the scalar value of the mass $m$, the gravitational constant $g$, and the discretization count $N$, then returns the force vector ${\bf f}$.

In [22]:
g = 9.81*10**2 #cm/s^2

In [26]:
#grade (enter your code in this cell - DO NOT DELETE THIS LINE) 
def get_force(m,g,N):
    # modify the force vector below and return it
    force = m*g*np.ones(N)
    return force

Now you can get the force vector `f` given `m`, `g`, and `N`:

In [27]:
f = get_force(m,g,N)
print(f)

[27795. 27795. 27795. 27795. 27795.]


### Evaluate the displacement vector

Obtain the response of the mass-spring system by solving the linear system of equations

$${\bf K}{\bf u} = {\bf f}$$

**Try this!**


First, solve the system of equations above by taking the inverse of the stiffness matrix using the ${\bf K}$ and ${\bf f}$. You can use [`numpy.linalg.inv`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html).

Store your results in the variable `u`

In [29]:
# define u below
u = f * np.linalg.inv(K)

**Try this!**


Solve the same system of linear equations using [`numpy.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html).
This function will take ${\bf K}$ and ${\bf f}$ and return ${\bf u}$.  
Store your results in the variable `u2`

In [30]:
# define u2 below
u2 = np.linalg.solve(K,f)

`u` and `u2` are the same! We will soon talk more about how to "break apart" what is happening inside `solve`.

### Vizualizing the deformation

<img src="two_springs.png" width=300>

We will create a numpy array `coord` with the position of each mass (the point that connects each spring of length $l$).  The position of the first mass is $0$ and the position of the last mass is $L$. The 1d numpy array `coord` has length `N+1`. The position $0$ corresponds to the mass $m_0$, which is the location where the system is getting constrained (held in place).

In [31]:
coord = np.array([l*i for i in range(N+1)])
coord

array([ 0.,  2.,  4.,  6.,  8., 10.])

Recall that the displacement $u$ obtained above corresponded to the movement of each mass. Check the shape of the array `coord` and the array `u`. What do you notice?

In [32]:
print(coord.shape, u.shape)

(6,) (5, 5)


**Try this!**


Hmm... What do you think you need to do next?  How can you include the initial condition at the wall? Create a numpy array `utotal` with the displacement of all the masses, that has the same shape of `coord`.  
Hint: you can use [`np.append(var, array)`](https://numpy.org/doc/stable/reference/generated/numpy.append.html) to add a single `var` at the beginning of `array`.

In [ ]:
# define utotal below
utotal = np.append()

Now, you can get the deformed position (coordinate) of the masses at the equilibrium (final) configuration using `coord` and `utotal`.

In [ ]:
def_coord = coord + utotal
def_coord

We provide a helper function to help you vizualize the final configuration of the slinky. You can use the code snippet below once you have created the arrays `coord` and `def_coord`.

In [ ]:
from helper_functions import draw_slinky
plt.figure()
plt.axis('equal')
draw_slinky(-5, coord, 2)
draw_slinky(5, def_coord, 2)

You should see in blue the original configuration of the slinky, and in orage the deformed configuration of the slinky!